Read Ramps. Get the wanted three columns

In [1]:
import pandas as pd
import json

# Load the JSON data into a pandas DataFrame
with open('../data/Ramps New.json') as f:
    data = json.load(f)
df = pd.json_normalize(data, 'features')

# Extract the first 5 scores and their coordinates
coordinate_score_df = df[['geometry.coordinates', 'properties.score_1', 'properties.FID']]

coordinate_score_df = coordinate_score_df.rename(columns={'geometry.coordinates': 'coordinates', 'properties.score_1': 'score_1'})
new_df = coordinate_score_df
# Print the result
# print(coordinate_score_df)

Create the latitude and longitude columns in the df amd add ramp points to the map.

Is coordinate_score_df used after?

In [2]:
import folium
import pyproj
project = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326')
coordinate_score_df['latitudes'], coordinate_score_df['longitudes'] = zip(*coordinate_score_df['coordinates'].apply(lambda coord: project.transform(coord[0], coord[1])))

m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# for _, row in coordinate_score_df.iterrows():
#     title = f"Score: {row['score_1']}"
#     folium.Marker(location=[row['latitudes'], row['longitudes']], popup=title).add_to(m)


# m.save('maps/Ramp_Scores_Map.html')

Use sjoin to combine the gdf.
Now the score_1 of shapes are still null

In [5]:
import geopandas as gpd
from shapely.geometry import Point

sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
points_gdf = gpd.GeoDataFrame(
    coordinate_score_df, 
    geometry=[Point(coord) for coord in coordinate_score_df['coordinates']]
)


pd.set_option('display.max_columns', None)
joined_gdf = gpd.sjoin(points_gdf, sidewalks_gdf, op='within')
avg_scores = joined_gdf.groupby('SWK_ID', as_index=False)['score_1'].mean()
sidewalks_gdf = sidewalks_gdf.set_index('SWK_ID')
avg_scores = avg_scores.set_index('SWK_ID')
sidewalks_gdf = sidewalks_gdf.merge(avg_scores, on='SWK_ID', how='left')
sidewalks_gdf = sidewalks_gdf.reset_index()
sidewalks_gdf = sidewalks_gdf[['geometry', 'score_1', 'SWK_ID']]
# print(sidewalks_gdf.columns)
sidewalks_gdf.info()
sidewalks_gdf.to_file('../data/Sidewalks_without_scores.json', driver='GeoJSON')


c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\xiao\AppData\Local\Temp\ipykernel_1992\3394290556.py:11: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:6492

  joined_gdf = gpd.sjoin(points_gdf, sidewalks_gdf, op='within')
c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23520 entries, 0 to 23519
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  23347 non-null  geometry
 1   score_1   0 non-null      float64 
 2   SWK_ID    23520 non-null  object  
dtypes: float64(1), geometry(1), object(1)
memory usage: 551.4+ KB


Add the original ramp points to an empty map of sjoined(?) sidewalk shapes.

Present Meaningless

In [6]:
from shapely.geometry import Point
import folium
import pandas as pd
import geopandas as gpd

#new_df: the df of ramps new.json(original)   Ramp points with scores

# Convert coordinates in new_df to same CRS as sidewalks file
new_df = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df['coordinates'].str[0], new_df['coordinates'].str[1]))
new_df.crs = {'init': 'epsg:6492'}  # replace with appropriate CRS

# Load sidewalks file as geopandas dataframe
sidewalks = gpd.read_file('../data/Sidewalks_without_scores.json')


# Create a map
m = folium.Map(location=[42.3601, -71.0589], zoom_start=30)

# Add sidewalk polygons to the map
folium.Choropleth(
    geo_data=sidewalks,
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    name='Sidewalks',
    highlight=True
).add_to(m)

# Add point markers for new_df to the map
for _, row in points_gdf.iterrows():
    title = f"Score: {row['score_1']}"
    folium.Marker(location=[row['latitudes'], row['longitudes']], popup=title).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('maps/test.html')


#print(sidewalks.crs)
#print(new_df.crs)
# Perform spatial join
#joined = gpd.sjoin(new_df, sidewalks, op='within')
#print(joined.columns)
#joined.info()
# Group by SWK_ID and calculate mean score_1
#result = joined.groupby('SWK_ID')['score_1'].mean()


c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


sidewalks_test.json is the same thing with ramps_without_scores.

In [559]:
sidewalks.to_file('sidewalks_test.json', driver='GeoJSON')
print(points_gdf.head())
sidewalks = gpd.read_file('sidewalks_test.json')

                                coordinates    score_1  properties.FID   
0   [-7911843.053800002, 5213855.430799998]  82.400000               0  \
1   [-7912104.434000002, 5213764.354100004]  92.800000               1   
2  [-7912111.2201999985, 5213780.467799999]  66.341154               2   
3    [-7912093.786999997, 5213788.37470001]  95.975000               3   
4   [-7911853.312899999, 5213875.931500003]  87.750000               4   

   latitudes  longitudes                          geometry  
0  42.352009  -71.073295  POINT (-7911843.054 5213855.431)  
1  42.351405  -71.075643  POINT (-7912104.434 5213764.354)  
2  42.351512  -71.075704  POINT (-7912111.220 5213780.468)  
3  42.351564  -71.075548  POINT (-7912093.787 5213788.375)  
4  42.352145  -71.073388  POINT (-7911853.313 5213875.932)  


Assign the nearest sidewalk to each point with a score. ID and geometry

In [561]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from scipy.spatial import KDTree

#Ramps New
ramps = points_gdf.head(10)
ramps = ramps.rename(columns={'properties.FID':'FID'})

sidewalks['score_1'] = None

# Create a new column in the ramps dataframe to store the nearest sidewalk
ramps['nearest_sidewalk'] = None

# Loop through each ramp and calculate the distance to each sidewalk
for ramp_index, ramp_row in ramps.iterrows():
    ramp_point = ramp_row['geometry']
    min_distance = float('inf')
    nearest_sidewalk_index = None
    for sidewalk_index, sidewalk_row in sidewalks.iterrows():
        sidewalk_point = sidewalk_row['geometry']
        distance = ramp_point.distance(sidewalk_point)
        if distance < min_distance:
            min_distance = distance
            nearest_sidewalk_index = sidewalk_index
            nearest_sidewalk_id = sidewalks.at[sidewalk_index, 'SWK_ID']
    
    # Store the index of the nearest sidewalk in the ramps dataframe
    ramps.at[ramp_index, 'nearest_sidewalk'] = int(nearest_sidewalk_id)


# Join the ramps dataframe with the sidewalks dataframe to get the geometry of the nearest sidewalk
ramps = ramps.join(sidewalks.set_index(sidewalks.index), on='nearest_sidewalk', rsuffix='_nearest')

In [562]:

print(ramps)

                                coordinates    score_1  FID  latitudes   
0   [-7911843.053800002, 5213855.430799998]  82.400000    0  42.352009  \
1   [-7912104.434000002, 5213764.354100004]  92.800000    1  42.351405   
2  [-7912111.2201999985, 5213780.467799999]  66.341154    2  42.351512   
3    [-7912093.786999997, 5213788.37470001]  95.975000    3  42.351564   
4   [-7911853.312899999, 5213875.931500003]  87.750000    4  42.352145   
5            [-7911529.5534999985, 5212385]  94.700000    5  42.342247   
6   [-7911551.325599996, 5212364.080200002]  99.600000    6  42.342108   
7   [-7911523.409899997, 5212374.193499997]  86.875000    7  42.342175   
8   [-7911539.937299997, 5212356.935199998]  86.787179    8  42.342060   
9   [-7912980.275599999, 5207269.563600004]  89.106667    9  42.308272   

   longitudes                          geometry nearest_sidewalk   
0  -71.073295  POINT (-7911843.054 5213855.431)            22774  \
1  -71.075643  POINT (-7912104.434 5213764.354)  

**A TRUE START**

Read the ramps_gdf and sidewalks_gdf again.

Create a buffer for points of polygon sides

In [17]:
ramps_gdf = gpd.read_file('../data/Ramps New.json')
sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
ramps_gdf = ramps_gdf.to_crs(sidewalks_gdf.crs)
ramps_gdf.geometry = ramps_gdf.geometry.buffer(5)

Sjoin the ramp points with the sidewalk shapes.

In [22]:
import json
import geojson
from shapely.geometry import shape, Point, Polygon
import geopandas as gpd


joined_gdf = gpd.sjoin(sidewalks_gdf, ramps_gdf, op='intersects')
grouped_gdf = joined_gdf.groupby('SWK_ID').agg({'FID_left': list, 'score_1': 'mean'}).reset_index()
result_gdf = sidewalks_gdf.merge(grouped_gdf, on='SWK_ID', how='left')
print(result_gdf.head())
print(result_gdf.dtypes)
result_gdf = result_gdf.drop(columns=['FID_left'])
result_gdf.to_file('../data/sidewalks_scores.json', driver='GeoJSON')



c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


   FID SWK_ID MATERIAL SWK_WIDTH          DISTRICT SWK_AREA  PARENT  \
0    0      1       CC       5.5       CHARLESTOWN     3648  ALBIO1   
1    1    402       CC         6  SOUTH DORCHESTER     1671  ALMON1   
2    2    403       CC         6  SOUTH DORCHESTER     1351  ALMON1   
3    3    404       CC         6  SOUTH DORCHESTER     1617  ALMON1   
4    4    405       CC         6  SOUTH DORCHESTER     3795  ALMON1   

        SEG_ID   SIDE  CG_ID  Rpr_Yr  New_SCI  Recon_Date  HPNETWORK  \
0     ALBIO1_0   LEFT  24842       0       85         NaN          0   
1  ALMON1_1749   LEFT    401       0       55         NaN          1   
2  ALMON1_2047  RIGHT    402       0       67         NaN          1   
3  ALMON1_2047   LEFT    403       0       81         NaN          1   
4  ALMON1_2338  RIGHT    404    2015       66         NaN          1   

   GOODCANDID                                           geometry FID_left  \
0           0  POLYGON ((772436.303 2964399.527, 772434.783 2..

Add the sidewalks with their colors onto the map.

In [14]:
import folium


new_gdf = result_gdf
# Create a map centered at a specific location
m = folium.Map(location=[42.361145, -71.057083], zoom_start=13)

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=new_gdf,
    name='Average Score',
    data=new_gdf,
    columns=['SWK_ID', 'score_1'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    reverse=True,
    highlight=True,
    show=False
).add_to(m)

#Why there are no geometry data
print(new_gdf['score_1'].isna().sum())
new_gdf = new_gdf.dropna(subset=['geometry'])

# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    new_gdf,
    name='Sidewalk Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID','score_1'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['SWK_ID','score_1'], labels=True, sticky=True),
    show=False
).add_to(m)


# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('maps/sidewalks_map.html')

4256


Group the scores by the neighborhood

In [24]:
neighborhoods_gdf = gpd.read_file('../data/Climate_Ready_Boston_Social_Vulnerability.geojson')
sidewalk_gdf = new_gdf.to_crs(neighborhoods_gdf.crs)
sidewalk_gdf = sidewalk_gdf.drop(columns=['FID_left'])



In [25]:
joined_gdf = gpd.sjoin(sidewalk_gdf, neighborhoods_gdf, how='left', op='within')
neighborhood_scores = joined_gdf.groupby('FID_right')['score_1'].mean().reset_index()
neighborhood_scores = neighborhood_scores.rename(columns={'FID_right': 'FID'})
neighborhoods_with_scores_gdf = neighborhoods_gdf.merge(neighborhood_scores, on='FID', how='left')
neighborhoods_with_scores_gdf = neighborhoods_with_scores_gdf.dropna(subset=['score_1'])
print(neighborhoods_with_scores_gdf)





c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


     FID      GEOID10   AREA_SQFT  AREA_ACRES  POP100_RE  HU100_RE  TotDis  \
0      1  25025010405  3914567.54     89.8661       5522       994     470   
1      2  25025010404  1472713.92     33.8089       5817      1862     299   
2      3  25025010801  1376667.12     31.6039       2783      1899      84   
3      4  25025010702  3228780.12     74.1226       2400      1643      45   
4      5  25025010204  2741497.18     62.9361       3173      1283     131   
..   ...          ...         ...         ...        ...       ...     ...   
175  176  25025081800  4024571.84     92.3915       2898      1388     329   
176  177  25025081700  6819156.25    156.5463       3820      1599     715   
177  178  25025081500  3619583.13     83.0942       2134       905     340   
178  179  25025081400  6678367.07    153.3142       3003      1462     268   
179  180  25025081300  6246271.61    143.3947       4760      2111    1146   

     TotChild  OlderAdult  Low_to_No   LEP  POC2  MedIllnes    

Add the color cover

In [26]:

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=neighborhoods_with_scores_gdf,
    name='Neighborhood Colors',
    data=neighborhoods_with_scores_gdf,
    columns=['FID', 'score_1'],
    key_on='feature.properties.FID',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    bins=list(range(0, 101, 10)),
    reverse=True,
    highlight=True,
    show=False
).add_to(m)

print(neighborhoods_with_scores_gdf['score_1'].isna().sum())


# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    neighborhoods_with_scores_gdf,
    name='Neighborhood Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['FID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['score_1', 'Name'], labels=True, sticky=True),
    show=False
).add_to(m)




# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('maps/neighborhood_scores.html')

0


Count the poor ramps as hazards

In [35]:
#hazards counts
df = pd.read_csv('../data/Sidewalk_Hazards_1.csv')
hazard_df = df[['FID', 'Type', 'x', 'y']]
badramps_gdf = gpd.read_file('../data/Ramps New.json')
sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
badramps_gdf = badramps_gdf.to_crs(sidewalks_gdf.crs)

In [73]:
index_to_drop = sidewalk_gdf.loc[sidewalk_gdf['SWK_ID'] == ' '].index
sidewalk_gdf.drop(index_to_drop, inplace=True)
sidewalk_gdf[sidewalk_gdf['SWK_ID']==' ']

,FID,SWK_ID,MATERIAL,SWK_WIDTH,DISTRICT,SWK_AREA,PARENT,SEG_ID,SIDE,CG_ID,Rpr_Yr,New_SCI,Recon_Date,HPNETWORK,GOODCANDID,geometry,score_1


Change CSV to a geojson for future combination

In [77]:
badramps_gdf = badramps_gdf[badramps_gdf['score_1'] <= 20]

# Add a suffix to the FID column name in the badramps_gdf GeoDataFrame
badramps_gdf = badramps_gdf.rename(columns={'FID': 'FID_ramp'})

# Add a suffix to the FID column name in the hazard_df DataFrame
hazard_df = hazard_df.rename(columns={'FID': 'FID_hazard'})

hazard_gdf = gpd.GeoDataFrame(hazard_df, geometry=gpd.points_from_xy(hazard_df.x, hazard_df.y))
hazard_gdf.crs = 'EPSG:4326'
hazard_gdf = hazard_gdf.to_crs(sidewalks_gdf.crs)
hazard_gdf.geometry = hazard_gdf.geometry.buffer(5)
badramps_gdf.geometry = badramps_gdf.geometry.buffer(5)
#buffer area
print(hazard_gdf.head())


   FID_hazard                                   Type          x          y  \
0           1  Fixed pinch point <36” sidewalk width -71.069782  42.381454   
1           2     Trip hazards not due to tree roots -71.056617  42.295022   
2           3     Trip hazards not due to tree roots -71.069829  42.381396   
3           4     Trip hazards not due to tree roots -71.061326  42.296611   
4           5     Trip hazards not due to tree roots -71.061093  42.293839   

                                            geometry  
0  POLYGON ((772408.347 2964324.571, 772408.323 2...  
1  POLYGON ((776128.777 2932845.452, 776128.753 2...  
2  POLYGON ((772395.756 2964303.352, 772395.732 2...  
3  POLYGON ((774851.878 2933418.132, 774851.854 2...  
4  POLYGON ((774920.250 2932408.223, 774920.226 2...  


Deal with ramp points. I want to change all the failure type to poor ramps

In [78]:
new_hazard_gdf = hazard_gdf.drop(['x', 'y'], axis=1)
new_hazard_gdf = new_hazard_gdf.rename(columns={'FID_hazard': 'FID'})


new_badramps_gdf = badramps_gdf[['FID_ramp', 'geometry','Failure_Ty']]
new_badramps_gdf = new_badramps_gdf.rename(columns={'FID_ramp': 'FID'})
new_badramps_gdf = new_badramps_gdf.rename(columns={"Failure_Ty": "Type"})
new_df_t = new_badramps_gdf
new_df_t.loc[new_df_t['Type'] == ' ', 'Type'] = 'Poor Ramp'
new_df_t['Type'].unique()
# new_df_t.info()
# new_badramps_gdf['Type'].isnull().sum()
# print(new_badramps_gdf.head())


array(['Concrete FailurePvt/Cswk Failure',
       'Concrete FailureDWP FailurePvt/Cswk Failure',
       'Concrete FailureDWP Failure', 'Missing/Depressed Dwy',
       'Concrete Failure', 'Poor Ramp'], dtype=object)

In [79]:
combined_gdf = pd.concat([new_badramps_gdf, new_hazard_gdf], ignore_index=True)
combined_gdf['Type'].unique()
# combined_gdf.to_csv('../data/combined_gdf.csv', index=False)
# print(combined_gdf.head())

array(['Concrete FailurePvt/Cswk Failure',
       'Concrete FailureDWP FailurePvt/Cswk Failure',
       'Concrete FailureDWP Failure', 'Missing/Depressed Dwy',
       'Concrete Failure', 'Poor Ramp',
       'Fixed pinch point <36” sidewalk width',
       'Trip hazards not due to tree roots',
       'Trip hazards due to tree roots'], dtype=object)

sjoin the sidewalks with all the hazards. fill null with 0. abandon null geometry. 

All sidewalks can't have the correct types that the merged ones have. We need use another combination way like ID to add the types manually.

In [80]:
hazard_intersections = gpd.sjoin(sidewalks_gdf, combined_gdf, op='intersects', how='left')
count_per_sidewalk = hazard_intersections.groupby('SWK_ID').size()
hazard_intersections = hazard_intersections.join(count_per_sidewalk.rename('count'), on='SWK_ID')
hazard_intersections['count'] = hazard_intersections['count'].fillna(0)
hazard_intersections = hazard_intersections.dropna(subset=['geometry'])
hazard_intersections = hazard_intersections.loc[:, ['SWK_ID', 'geometry', 'count','Type']]
print(hazard_intersections.info())
# Group the rows by SWK_ID and apply a function to each group
grouped = hazard_intersections.groupby('SWK_ID').apply(lambda x: [s for s in x['Type'].astype(str).tolist() if s != 'nan'])
hazard_intersections['Type_list'] = grouped.rename('Type').reset_index(drop=True)


c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 27292 entries, 0 to 23519
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   SWK_ID    27292 non-null  object  
 1   geometry  27292 non-null  geometry
 2   count     27292 non-null  int64   
 3   Type      10577 non-null  object  
dtypes: geometry(1), int64(1), object(2)
memory usage: 1.0+ MB
None


In [89]:
# print(hazard_intersections)
# sidewalks_gdf.info()
# print(hazard_intersections['Type_list'].unique())
# hazard_intersections_new = hazard_intersections.loc[:, ['SWK_ID','count', 'Type_list','geometry']]
# hazard_intersections_new.to_csv('../data/typeList.csv')
hazard_intersections['count'].unique()

array([ 2,  1,  6,  3,  4,  5,  7,  9,  8, 13, 12, 10, 14, 16, 15, 11, 17],
      dtype=int64)

In [88]:
index_to_drop = hazard_intersections.loc[hazard_intersections['SWK_ID'] == ' '].index
hazard_intersections.drop(index_to_drop, inplace=True)
hazard_intersections[hazard_intersections['SWK_ID']==' ']

,SWK_ID,geometry,count,Type,Type_list


In [90]:


m2 = folium.Map(location=[42.361145, -71.057083], zoom_start=13)

folium.Choropleth(
    geo_data=hazard_intersections,
    name='Hazard Counts',
    data=hazard_intersections,
    columns=['SWK_ID', 'count'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu_r',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Hazard Counts',
    highlight=True,
    show=False,
    bins=[0, 2, 4, 5, 6, 20]
).add_to(m2)




# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    hazard_intersections,
    name='Hazard Counts',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['count'], labels=True, sticky=True),
    show=False
).add_to(m2)

# Add a layer control to the map
folium.LayerControl().add_to(m2)

# Save the map as an HTML file
m2.save('maps/All_hazard_count_xiao.html')

In [580]:
null_sidewalks_gdf = result_gdf
null_sidewalks_gdf = null_sidewalks_gdf[null_sidewalks_gdf['score_1'].isna()]
null_sidewalks_gdf = null_sidewalks_gdf.dropna(subset=['geometry'])
print(null_sidewalks_gdf)
null_sidewalks_gdf.to_file("Sidewalks_With_No_Ramps.json", driver='GeoJSON')




         FID SWK_ID MATERIAL SWK_WIDTH          DISTRICT SWK_AREA  PARENT   
6          6    407       CC         6  SOUTH DORCHESTER     1854  ALMON1  \
13        13    414       CC         6  SOUTH DORCHESTER     3472  ASTOR1   
26        26    428       CC         7  SOUTH DORCHESTER     5461  ADAMS7   
27        27    429       CC         0  SOUTH DORCHESTER     2866  ADAMS7   
29        29    431       CC         0  SOUTH DORCHESTER     2040  ADAMS7   
...      ...    ...      ...       ...               ...      ...     ...   
23396  23396  18555       CC         6  SOUTH DORCHESTER     4711  STAND1   
23415  23415  23157       CC         6  SOUTH DORCHESTER     2334  DOONE1   
23416  23416  23157       CC         6  SOUTH DORCHESTER     2334  DOONE1   
23517  23517  21675       CC         6      WEST ROXBURY     1910  SPRIN7   
23518  23518   5596       CC       4.5         NORTH END     1277  CHART1   

            SEG_ID   SIDE  CG_ID  Rpr_Yr  New_SCI    Recon_Date  HPNETWORK 

In [581]:
m1 = folium.Map(location=[42.361145, -71.057083], zoom_start=13)

folium.Choropleth(
    geo_data=null_sidewalks_gdf,
    name='Null Sidewalks',
    data=null_sidewalks_gdf,
    columns=['SWK_ID', 'score_1'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu_r',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Null Sidewalks',
    highlight=True,
    show=False
    
).add_to(m1)




# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    null_sidewalks_gdf,
    name='Null Sidewalks',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['score_1'], labels=True, sticky=True),
    show=False
).add_to(m1)

# Add a layer control to the map
folium.LayerControl().add_to(m1)

# Save the map as an HTML file
m1.save('Sidewalks_With_No_Ramps.html')